In [1]:
# چند روش برای اسپارس کردن ماتریس

In [3]:
import scipy
import random
import portpy.photon as pp
import numpy as np
from cvxpy import *
from scipy import sparse
import sklearn.metrics
import cProfile
import time
import sys
import psutil
from sklearn.decomposition import PCA

In [4]:
data_dir = r'../data'
data = pp.DataExplorer(data_dir=data_dir)
patient_id = 'Lung_Phantom_Patient_1'
data.patient_id = patient_id
ct = pp.CT(data)
structs = pp.Structures(data)
beams = pp.Beams(data)
opt_params = data.load_config_opt_params(protocol_name='Lung_2Gy_30Fx')
structs.create_opt_structures(opt_params)
inf_matrix_sparse = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams)
protocol_name = 'Lung_2Gy_30Fx'
clinical_criteria = pp.ClinicalCriteria(data, protocol_name)
plan_sparse = pp.Plan(ct, structs, beams, inf_matrix_sparse, clinical_criteria)
opt = pp.Optimization(plan_sparse, opt_params=opt_params)
opt.create_cvxpy_problem()

sol_sparse = opt.solve(solver='MOSEK', verbose=True)
dose_sparse_1d = plan_sparse.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions())
x_sparse =sol_sparse['optimal_intensity'] * plan_sparse.get_num_of_fractions()

beams_full = pp.Beams(data, load_inf_matrix_full=True)
inf_matrix_full = pp.InfluenceMatrix(ct=ct, structs=structs, beams=beams_full, is_full=True)
plan_full = pp.Plan(ct, structs, beams, inf_matrix_full, clinical_criteria)
dose_full_1d = plan_full.inf_matrix.A @ (sol_sparse['optimal_intensity'] * plan_full.get_num_of_fractions())

A_full = plan_full.inf_matrix.A
A_sparse = plan_sparse.inf_matrix.A


creating rinds.. This step may take some time due to dilation
rinds created!!
Creating BEV..
Loading sparse influence matrix...
Done
Objective Start
Objective done
Constraints Start
Structure ESOPHAGUS not available!
Structure ESOPHAGUS not available!
Constraints done
                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Nov 10 03:30:52 PM: Your problem has 1946 variables, 14 constraints, and 0 parameters.
(CVXPY) Nov 10 03:30:52 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 10 03:30:52 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 10 03:30:52 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 10 03:30:52 PM: Your problem is compiled with the CPP canonicalization backend.
----------------------------------------

(CVXPY) Nov 10 03:30:55 PM:   Dual.    obj: 4.2045433929e+01    nrm: 6e+03    Viol.  con: 0e+00    var: 2e-08    cones: 0e+00  
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Nov 10 03:30:55 PM: Problem status: optimal
(CVXPY) Nov 10 03:30:55 PM: Optimal value: 4.205e+01
(CVXPY) Nov 10 03:30:55 PM: Compilation took 1.057e+00 seconds
(CVXPY) Nov 10 03:30:55 PM: Solver (including time spent in interface) took 1.937e+00 seconds
Creating BEV..
Loading full influence matrix..
Done


In [7]:
nonzero_count = np.count_nonzero(A_full)
print("تعداد مقادیر غیر صفر ماتریس کامل: ", nonzero_count)
print("تعداد مقادیر غیر صفر ماتریس پراکنده: ", A_sparse.getnnz())

تعداد مقادیر غیر صفر ماتریس کامل:  10279072
تعداد مقادیر غیر صفر ماتریس پراکنده:  335573


In [9]:
from scipy.sparse import coo_matrix

A_sparse_coo = coo_matrix(A_full)

print(A_sparse_coo.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
print("تعداد مقادیر غیر صفر ماتریس پراکنده: ", A_sparse_coo.getnnz())

تعداد مقادیر غیر صفر ماتریس پراکنده:  10279072
